In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install huggingface_hub

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''


In [7]:
from langchain.llms import OpenAI
from langchain import HuggingFaceHub, LLMChain, PromptTemplate
import openai

In [46]:
# models = openai.Model.list()
# print(models)


# llm = OpenAI(model='text-davinci-003',temperature=0.9)
# text = "What would be a good company name for a company that makes colorful socks?"
# print(llm(text))

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

def transform_llm_result(generations):
    answers = []
    for generation in generations:
        answer = generation[0].text
        answers.append(answer)
    print(answers)
    return answers


In [52]:

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-xl',
    model_kwargs={'temperature':1e-10}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# initialize the models
davinci = OpenAI(
    model_name="text-davinci-003"
)

davinci_llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

In [61]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]

def ask_llm(chain, qs):
    result = chain.generate(qs)
    generations = result.generations
    return transform_llm_result(generations)
    
output = ask_llm(llm_chain, qs)
op = ask_llm(davinci_llm_chain, qs)

#compare the results
if op == output: 
    print ("Davinci and Flan have same answers") 
else: 
    print ("Davinci and Flan have different answers") 

['green bay packers', '184', 'john glenn', 'one']
[' The Green Bay Packers won the Super Bowl in the 2010 season.', ' 6 ft 4 inches is equal to 193.04 centimeters.', ' Eugene A. Cernan was the 12th person on the moon. He was part of the Apollo 17 mission, which launched on December 7, 1972.', ' A blade of grass does not have eyes.']
Davinci and Flan have different answers
